# Table of Contents
1  Image概述
2  Image operator(mask, clip, select, addBands, export)
3  ImageCollection概述
4  ImageCollection operator(filter, select, set/get, calculation, toList)
5  Example 1. Calculate NDVI for Jiangxi and Extract NDWI for poyang Lake between 1982 to 2021;
6  Example 2. Download huge ImageCollection from google earth engine; -> Download Images
7  Example 3. Extract NDVI values of climate stations from 1982 to 2021; -> Spatial analysis
8  Example 4. Supervised Classification -> Images Classification
9  Example 5. Unsupervised Classification (clustering) 

In [1]:
import ee, os
import geemap
# ee.Initialize()
Map = geemap.Map()
Map = geemap.Map(center=(39, 110), zoom=4)
Map

Map(center=[39, 110], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(To…

In [ ]:
image = ee.Image('LANDSAT/LC08/C02/T1_TOA/LC08_044034_20140318').select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7'])


In [ ]:
# // Make an Array Image, with a 1-D Array per pixel.
arrayImage1D = image.toArray()
# arrayImage1D.shape

In [ ]:
# // Make an Array Image with a 2-D Array per pixel, 6x1.
arrayImage2D = arrayImage1D.toArray(1)

In [2]:
# -------------------------------------------------------
# 2  ImageCollection 去云
#  Calculate NDVI for Jiangxi
# -------------------------------------------------------
year = 2010, 2010
doy = '-01-01', '-12-31'

start_date = str(year[0]) + doy[0]
end_date = str(year[1]) + doy[1]

jx_bound = ee.FeatureCollection('users/yehuigeo/jx_bound')

In [3]:
def cloudMaskL457(image):
    qaMask = image.select('QA_PIXEL').bitwiseAnd(int('11111', 2)).eq(0)
    saturationMask = image.select('QA_RADSAT').eq(0)
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2)
    thermalBand = image.select('ST_B6').multiply(0.00341802).add(149.0)

    return image.addBands(opticalBands, None, True)\
                 .addBands(thermalBand, None, True)\
                 .updateMask(qaMask).updateMask(saturationMask)\
                 .copyProperties(image, ["system:time_start",'system:id'])

def maskS2clouds(image):   # This function was used to mask the clouds for sentinel-2
    qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(
             qa.bitwiseAnd(cirrusBitMask).eq(0))

  # Return the masked and scaled data, without the QA bands.
    return image.updateMask(mask).divide(10000).select("B.*").copyProperties(image, ["system:time_start",'system:id'])

def maskL8sr(image):
    qaMask = image.select('QA_PIXEL').bitwiseAnd(int('11111',2)).eq(0);
    saturationMask = image.select('QA_RADSAT').eq(0);

#   // Apply the scaling factors to the appropriate bands.
    opticalBands = image.select('SR_B.').multiply(0.0000275).add(-0.2);
    thermalBands = image.select('ST_B6').multiply(0.00341802).add(149.0);
    
    return image.addBands(opticalBands, None, True)\
                 .addBands(thermalBands, None, True)\
                 .updateMask(qaMask)\
                 .updateMask(saturationMask)\
                 .copyProperties(image, ["system:time_start",'system:id'])

In [4]:
# Landsat 3 MSS Collection 1 Tier 1 Raw Scenes
# Dataset Availability
# 1978-03-05 - 1983-03-31
# Landsat 5+7 scenes
# Dataset Availability
# 1984 - 2012
# get landsat 5
collection5 = ee.ImageCollection('LANDSAT/LT05/C02/T1_L2') \
    .filterBounds(jx_bound) \
    .filterDate(start_date, end_date) \
    .map(cloudMaskL457) \
    .select(['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7'])
# print('Landsat5')
# print(collection5.size().getInfo())
# Landsat 7 Collection 1 Tier 1 Raw Scenes
# Dataset Availability
# 1999 - 2021
# get landsat 7
# https://developers.google.com/earth-engine/datasets/catalog/landsat
# https://www.usgs.gov/landsat-missions/landsat-collection-2
collection7 = ee.ImageCollection('LANDSAT/LE07/C02/T1_L2') \
    .filterBounds(jx_bound) \
    .filterDate(start_date, end_date) \
    .map(cloudMaskL457) \
    .select(['SR_B1', 'SR_B2', 'SR_B3', 'SR_B4', 'SR_B5', 'SR_B7'])

# print(collection7.size().getInfo())

In [5]:
# Landsat 8 Collection 1 Tier 1 Raw Scenes
# Dataset Availability
# 2013 - 2021
# get landsat 8
collection8 = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')\
    .filterBounds(jx_bound) \
    .filterDate(start_date, end_date) \
    .map(maskL8sr).select(['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'pixel_qa'],['B1', 'B2', 'B3', 'B4', 'B5', 'B7', 'pixel_qa']) 
# print(collection8.size().getInfo())
# Sentinel-2 (S2) Multispectral Instrument (MSI) 
# Dataset Availability
# Level-1C orthorectified top-of-atmosphere reflectance -> 2015-06-23 - present
# Level-2A orthorectified atmospherically corrected surface reflectance.
# get sentinel-2A
sentinel2 = ee.ImageCollection("COPERNICUS/S2_SR").filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20))\
    .filterDate(start_date, end_date).map(maskS2clouds).filterBounds(jx_bound)\
    .select(['B2', 'B3', 'B4', 'B8', 'B11', 'B12'],['B1', 'B2', 'B3', 'B4', 'B5', 'B7']) 
# print(sentinel2.size().getInfo())

In [6]:
def vfc(img):
    ndvi = img.normalizedDifference(['SR_B4','SR_B3']).rename('NDVI')
    xmin = ee.Number(ndvi.reduceRegion(**{
          'reducer': ee.Reducer.min(),
          'geometry': img.geometry(),
          'scale': 30,
          'maxPixels': 1e13
        }).get('NDVI'))
    xmax = ee.Number(ndvi.reduceRegion(**{
          'reducer': ee.Reducer.max(),
          'geometry': img.geometry(),
          'scale': 30,
          'maxPixels': 1e13
        }).get('NDVI'))
                          
    fv = ndvi.expression('(ndvi - xmin) / (xmax - xmin)',{
            'ndvi': ndvi,
            'xmin': xmin,
            'xmax': xmax
            }).rename('vfc')
#     fv = ndvi.subtract(xmin).divide(xmax.subtract(xmin)).rename('vfc')
    return img.addBands(fv)


In [ ]:
xmin.getInfo()

In [10]:
# ndvi = collection7.first().normalizedDifference(['SR_B4','SR_B3']).rename('NDVI')
xmin = ee.Number(ndvi1.reduceRegion(**{
      'reducer': ee.Reducer.min(),
      'geometry': roi,
      'scale': 30,
      'maxPixels': 1e13
    }).get('NDVI'))
xmax = ee.Number(ndvi1.reduceRegion(**{
      'reducer': ee.Reducer.max(),
      'geometry': roi,
      'scale': 30,
      'maxPixels': 1e13
    }).get('NDVI'))

In [16]:
temp = ee.List([xmin, xmax])

In [17]:
temp.filter(ee.Filter.notNull())

AttributeError: 'ApiFunction' object has no attribute 'name'

In [9]:
roi = jx_bound.geometry()
scale = 30
img = collection5.first()
ndvi1 = img.normalizedDifference(['SR_B4','SR_B3']).rename('NDVI')
# col7_fv = collection7.map(vfc)

In [ ]:
collection5.aggregate_array

In [ ]:
a = collection5.first().geometry()
Map.addLayer(a, {}, 'a')

In [ ]:
col_fv = collection5.map(vfc)

In [ ]:
palette = ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718',
               '74A901', '66A000', '529400', '3E8601', '207401', '056201',
               '004C00', '023B01', '012E01', '011D01', '011301']
Map.addLayer(col_fv.select('vfc').mean().clip(roi),{min: 0, max: 1,'palette': palette}, 'fv2')

In [ ]:
# fv = vfc(collection7.first())
# fv = ndvi.subtract(xmin).divide(xmax.subtract(xmin)).rename('vfc')
fv1 = ndvi1.expression('(ndvi - xmin) / (xmax - xmin)',{
            'ndvi': ndvi1,
            'xmin': xmin,
            'xmax': xmax
            }).rename('vfc')
palette = ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718',
               '74A901', '66A000', '529400', '3E8601', '207401', '056201',
               '004C00', '023B01', '012E01', '011D01', '011301']
Map.addLayer(fv1,{min: 0, max: 1,'palette': palette}, 'fv1')

In [ ]:
fv = ndvi1.subtract(xmin).divide(xmax.subtract(xmin)).rename('FV')
palette = ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718',
               '74A901', '66A000', '529400', '3E8601', '207401', '056201',
               '004C00', '023B01', '012E01', '011D01', '011301']
Map.addLayer(fv.clip(roi),{min: 0, max: 1,'palette': palette}, 'fv')

In [ ]:
Map.addLayer(ndvi1,{min: 0, max: 1}, 'ndvi1')

In [ ]:
palette = ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718',
               '74A901', '66A000', '529400', '3E8601', '207401', '056201',
               '004C00', '023B01', '012E01', '011D01', '011301']
roi = jx_bound.geometry()
Map.addLayer(col7_fv.first().select('vfc').clip(roi),{min: 0, max: 1,'palette': palette}, 'VFC')

In [ ]:
xmin = ndvi.reduceRegion(**{
  'reducer': ee.Reducer.mean(),
  'geometry': roi,
  'scale': 30,
  'maxPixels': 1e13
})

In [ ]:
xmin.getInfo().get('NDVI')

In [ ]:
xmin.getInfo()

In [ ]:
# Map.setCenter(29.50, 116,zoom=4)
Map.addLayer(sentinel2.median().clip(jx_bound),{'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.3},'ly2020')
Map.addLayer(sentinel2.max().clip(jx_bound),{'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 0.3},'ly2022')
# Map.split_map(left_layer='ly2020', right_layer='ly2022')

In [ ]:
# -------------------------------------------------------
# 3  ImageCollection 
#  Calculate NDVI for Jiangxi
# Bands math operator by image expression
# https://code.earthengine.google.com/2dd2d4e898e633ea5c0c78066f74d1b7?accept_repo=users%2Fnclinton%2FEE101
# -------------------------------------------------------
# function for EVI,GCVI,NDVI,NDWI,LSWI
def f_EVI(img):
    EVI = img.expression(
        '2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', {
          'NIR': img.select('B4'),
          'RED': img.select('B3'),
          'BLUE':img.select('B1')})
    return EVI
def f_GCVI(img):
    GCVI = img.expression(
        '(NIR / Green - 1)', {
          'NIR': img.select('B4'),
          'Green': img.select('B2')})
    return GCVI
def addVIs(img):
    NDVI = img.normalizedDifference(['B4','B3']).rename('NDVI')
    mNDWI = img.normalizedDifference(['B2','B5']).rename('mNDWI')
    LSWI = img.normalizedDifference(['B5','B6']).rename('LSWI')
    EVI = f_EVI(img)
    GCVI = f_GCVI(img)
    return img.addBands(NDVI).addBands(NDWI).addBands(LSWI).addBands(EVI).addBands(GCVI)

In [ ]:
sentinel2 = ee.ImageCollection("COPERNICUS/S2_SR").filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20))\
    .filterDate(start_date, end_date).map(maskS2clouds).filterBounds(jx_bound)\
    .select(['B2', 'B3', 'B4', 'B8', 'B11', 'B12'],['B1', 'B2', 'B3', 'B4', 'B5', 'B7']).map(addVIs) 

In [ ]:
# https://code.earthengine.google.com/c41d27de2bf046ef2cccc75019816497?accept_repo=users%2Fnclinton%2FEE101
# https://github.com/HuiYeAmazing/Study-notes/blob/master/python/jupyter/geemap/Image/03_Image%E5%8F%AF%E8%A7%86%E5%8C%96%E5%8F%82%E6%95%B0%E4%BA%86%E8%A7%A3.md
# ---------------------------
# CSS stands for colours
# https://www.quackit.com/css/tutorial/css_introduction.cfm
# https://www.quackit.com/css/color/charts/css_color_names_chart.cfm
# -----------------------------
ndvi = sentinel2.first().select('NDVI')
palette = ['FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718',
               '74A901', '66A000', '529400', '3E8601', '207401', '056201',
               '004C00', '023B01', '012E01', '011D01', '011301']
Map.addLayer(ndvi, {min: 0, max: 1, palette: palette}, 'NDVI');